In [ ]:
#Import Packages
#Needed for moving to output
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as pl
import itertools
from collections import Counter
import pickle

# Install tumor-tcell model

In [ ]:
!pip install tumor-tcell

#Import other needed packages
from vivarium.library.units import units, remove_units
from tumor_tcell.experiments.main import tumor_tcell_abm
from tumor_tcell.experiments.main import plots_suite
from tumor_tcell.experiments.main import get_tcells
from tumor_tcell.experiments.main import get_tumors
from tumor_tcell.library.phylogeny import get_phylogeny

#Analysis tumor-tcell modules needed
from tumor_tcell.library.population_plots import death_group_plot
from tumor_tcell.library.population_plots import population_group_plot
from tumor_tcell.library.individual_analysis import individual_analysis

#Run a simulation
Our model is much more in depth than a single function, but you can take a look at all the code and organization and underlying processes at our GitHub site here: https://github.com/vivarium-collective/tumor-tcell



### Use the function to run a simulation
We can take the function we built for specifying a number of different parameters that wraps all the model together to simulate the tumor microenvironment.

It can take a long time to run with many cells, so while we are testing things out we will run it with fewer cells for longer times or more cells for shorter times. Right now it is set up as running with few cells for medium amount of times (not so interesting biologically).

This current set up should take about 3 minutes to run on Colab.

All parameters are critical to the modeling outputs and depend on the question you are trying to answer. However, the parameters we will most play around with are:

* ratio of tumor to T cells
* ratio of PD1+ and PD1- T cells
* total number of cells
* time of the simulation
* presence of DCs or LN process

In [ ]:
all_dir =  './data_modeling/ABM'
experiment_name = '20250303_ABMsimulation'
exp_id = '/' + '60000seconds_132cells'

#Make a directory to save simulation data
exper_dir = all_dir + '/' + experiment_name
if not os.path.exists(exper_dir):
      os.makedirs(exper_dir)

outdir = exper_dir + exp_id

######################################
#####SETTINGS FOR FULL EXPERIMENT#####
FULL_BOUNDS = [1200*units.um, 1200*units.um] #size of area for simulation
TIMESTEP = 60 #60 second intervals
NBINS = [120, 120] # how to segment the area for diffusion, etc.

data = tumor_tcell_abm(

    halt_threshold=5000,  # stop simulation at this number of cells/agents

    #Cell set up
    n_tumors=120, #number of tumors in simulation
    tumors_state_PDL1n=0.5, #percent of tumors that start PDL1-
    n_tcells=12, #number of T cells in simulation
    tcells_total_PD1n=9, #number of T cells out of total that are PD1-
    tcells_state_PD1n=None, #Set absolute amount above (could set percent but increase variation with replicates)

    #Lymph node options
    lymph_nodes=False, #whether or not you want to use LN process
    dendritic_state_active=0.5, #Assume half are in LN (active means they go to LN)
    n_dendritic=0, #number of dendritic cells in tumor
    n_tcells_lymph_node=0, #number of T cells that are in the LN

    #field
    bounds=FULL_BOUNDS, #size of area for simulation
    n_bins=NBINS, # how to segment the area for diffusion, etc.
    depth = 15, # um for calculating volumes for diffusion
    field_molecules=['IFNg'], #molecules that we use, if using LN then include 'tumor_debris'

    #Historic not needed
    tumors=None, #can import CODEX data
    tcells=None, #can import CODEX data
    dendritic_cells=None, #can import CODEX data

    #time
    total_time=60000, #total time of simulation in seconds
    time_step=TIMESTEP, #timestep intervals for updating simulation
    sim_step=100*TIMESTEP,  # simulation increments at which halt_threshold is checked
    emit_step=10*TIMESTEP, # simulation increments at which data is emitted for analysis
    emitter='timeseries', # how to save the data
    parallel=False, # whether you could do parallel processing

    #Placement of cells
    tumors_distance=260 * units.um,  # sqrt(n_tumors)*15(diameter)/2 can be used to determin based on number of tumors
    tcells_distance=220 * units.um,  # in (less than tumors_distance) or out (None) of the tumor
    tumors_excluded_distance=None, # if tumors need to be restricted radially
    tcells_excluded_distance=None,  # for creating a ring around tumor to say they are not enriched within tumors
    tumors_center=None, #
    tcell_center=None,

)

############Set parameters for plotting/saving the data#################
data = remove_units(data)
bounds = data[0.0]['tumor_environment']['dimensions']['bounds']

if not os.path.exists(outdir):
    os.makedirs(outdir)
data_export = open(outdir+'/data_export.pkl', 'wb')
pickle.dump(data, data_export)
data_export.close()

individual_analysis(analysis_dir = exper_dir, experiment_id=exp_id,
                    bounds=bounds, tcells=True, lymph_nodes=False)

# Analyze results

We generate many figures automatically from the results. These include at a single-cell level (first set of figures), spatial positions at different snapshots (second set of figures), and then population trends over time (last set of figures).

These are all saved as plots in the folder you created (e.g., '/data_modeling/ABM' above) as well as a video that was not able to be played here in the ouptut.

The raw modeling output data that tracks cells at a single cell level, on which all of this output was generated is also saved as .csv files.

We can think about all the different things that we could analyze like it would be a real experiment - there are many ways to interpret the data.

# Run analysis on CODEX data

We can directly import CODEX data that has x, y position, protein expression information, and cell type data that can be used as initial points for our simulation.

I have already eliminated all the non-essential data (protein columns and metadata) we do not need so that it is quick to upload the data and import into the simulation. Look at the columns that are being used here and each column is kept because it is used as a parameter in the model on a per cell type or per location basis.

This data is stored in the class Box that is shared under "Data" folder.

In [ ]:
#Read in the data from parent directory
analysis_file = '/home/john-hickey/Desktop/Lab Member Project/Bobby_Ni/Bobby Spatial Omics/2HCTcell_ABM_CODEX.csv'
experiment_id = '/2HC/'
experiment_dir = all_dir+experiment_id

if not os.path.exists(experiment_dir):
  os.makedirs(experiment_dir)

#Read in the data from parent directory
df_2HC_exp = pd.read_csv(analysis_file)
df_2HC_exp.rename(columns = {"Unnamed: 0": "cell_index"},  inplace = True)
df_2HC_exp.set_index('cell_index', inplace = True)
df_2HC_exp

Lets look at the area of the tissue that we decided to simulate (we took only a small portion of the tissue) rather than the whole tissue (it would take too long or be too computationally expensive).

In [ ]:
# Plotting
pl.scatter(df_2HC_exp['AbsoluteX'], df_2HC_exp['AbsoluteY'])
pl.title('Tissue Graph')
pl.xlabel('X-axis Label')
pl.ylabel('Y-axis Label')
pl.grid(True)
pl.show()

Notice that it is starting at point (0, 0). We will add some buffer so that the simulated tumor starts in the middle of the simulation environment (see that total bounds is going to be 1800).

In [ ]:
df_2HC_exp['AbsoluteX'] = df_2HC_exp['AbsoluteX'] + 600
df_2HC_exp['AbsoluteY'] = df_2HC_exp['AbsoluteY'] + 600
# Plotting
pl.scatter(df_2HC_exp['AbsoluteX'], df_2HC_exp['AbsoluteY'])
pl.title('Tissue Graph')
pl.xlabel('X-axis Label')
pl.ylabel('Y-axis Label')
pl.grid(True)
pl.show()

#Format the data for the Simulation
The data input into the simulation requires nested dictionaries as opposed to dataframes, so we need to reformat the data accordingly based on each condition.

In [ ]:
#Set bounds
bounds = [1800,1800]

#Create a copy
df_2HC_exp1 = df_2HC_exp.copy()

#Column Number for iloc iterating
Ypos = 0
Xpos = 1
H2Kb = 3
PD1 = 4
PDL1 = 5
Celltype = 7
IFNg_con = 8

tumor_diameter = 15 #set diamater for cell
tcell_diameter = 7.5 #set diamater for cell

tumors = {}
IFNg = {}
tcells = {}

for i in range(len(df_2HC_exp1)):
  if df_2HC_exp1.iloc[i, Celltype] == 'PDL1p' or df_2HC_exp1.iloc[i, Celltype] == 'PDL1n':
    tumors.update({
        'tumor_'+ str(df_2HC_exp1.index[i]) : {
            'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
            'type': 'tumor',
            'cell_state': df_2HC_exp1.iloc[i, Celltype],
            'PDL1': df_2HC_exp1.iloc[i, PDL1],
            'MHCI': df_2HC_exp1.iloc[i, H2Kb],
            'diameter': tumor_diameter*units.um,
        }
    })

    IFNg.update({
        df_2HC_exp1.index[i] : {
            'amount': df_2HC_exp1.iloc[i, IFNg_con],
            'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
        }
    })

  else:
    tcells.update({
        'tcell_'+ str(df_2HC_exp1.index[i]) : {
            'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
            'type': 'tcell',
            'cell_state': df_2HC_exp1.iloc[i, Celltype],
            'PD1': df_2HC_exp1.iloc[i, PD1],
            'diameter': tcell_diameter*units.um,
        }
    })
    IFNg.update({
        df_2HC_exp1.index[i]: {
        'amount': df_2HC_exp1.iloc[i, IFNg_con],
        'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
        }
    })

# print the first tcell
tcell_ids = list(tcells.keys())
print(tcells[tcell_ids[0]])

Now we can use the reformatted dictionaries as input to the same function we used above to simulate what might happen in the tumor from this CODEX data. Notice we do not need to specify as many parameters since they are coming from our data directly.

You can let this run as long as you want, but for the sake of running in class I have shortened to 600s (not very biologically relevant). You can increase further if you leave your notebook running. This should take about 1.5 minutes to run. It will largely scale linearly with the simulation time if you increase.

In [ ]:
#using experimental CODEX data
data = tumor_tcell_abm(tumors=tumors,
                       tcells=tcells,
                       total_time=600,
                       bounds=[b*units.um for b in bounds],
                       halt_threshold=3000,
                       emit_step=600,
                       field_molecules=['IFNg'],
                       emitter='timeseries',
                       lymph_nodes=False,)
data;

Use a similar code to analyze the data post-experiment

In [ ]:
############Set parameters for plotting/saving the data#################
data = remove_units(data)
#bounds = data[0.0]['tumor_environment']['dimensions']['bounds']


data_export = open(experiment_dir+'/data_export.pkl', 'wb')
pickle.dump(data, data_export)
data_export.close()

individual_analysis(analysis_dir = all_dir, experiment_id=experiment_id,
                    bounds=bounds, tcells=True, lymph_nodes=False)

#Assignment

### Problem #1 (50 points)

Run some simulations and change at least one important parameter. Comparing the results, what are some things that you observe from the simulation?

In [ ]:
df_2HC_exp1 = df_2HC_exp.copy()
df_2HC_exp1['AbsoluteX'] = df_2HC_exp1['AbsoluteX']
df_2HC_exp1['AbsoluteY'] = df_2HC_exp1['AbsoluteY']

In [ ]:
tumors = {}
IFNg = {}
tcells = {}

for i in range(len(df_2HC_exp1)):
  if df_2HC_exp1.iloc[i, Celltype] == 'PDL1p' or df_2HC_exp1.iloc[i, Celltype] == 'PDL1n':
    tumors.update({
        'tumor_'+ str(df_2HC_exp1.index[i]) : {
            'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
            'type': 'tumor',
            'cell_state': df_2HC_exp1.iloc[i, Celltype],
            'PDL1': df_2HC_exp1.iloc[i, PDL1],
            'MHCI': df_2HC_exp1.iloc[i, H2Kb],
            'diameter': tumor_diameter*units.um,
        }
    })

    IFNg.update({
        df_2HC_exp1.index[i] : {
            'amount': df_2HC_exp1.iloc[i, IFNg_con],
            'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
        }
    })

  else:
    tcells.update({
        'tcell_'+ str(df_2HC_exp1.index[i]) : {
            'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
            'type': 'tcell',
            'cell_state': df_2HC_exp1.iloc[i, Celltype],
            'PD1': df_2HC_exp1.iloc[i, PD1],
            'diameter': tcell_diameter*units.um,
        }
    })
    IFNg.update({
        df_2HC_exp1.index[i]: {
        'amount': df_2HC_exp1.iloc[i, IFNg_con],
        'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
        }
    })

# print the first tcell
tcell_ids = list(tcells.keys())
print(tcells[tcell_ids[0]])

In [ ]:
#using experimental CODEX data
data = tumor_tcell_abm(tumors=tumors,
                       tcells=tcells,
                       total_time=600,
                       bounds=[b*units.um for b in bounds],
                       halt_threshold=3000,
                       emit_step=600,
                       field_molecules=['IFNg'],
                       emitter='timeseries',
                       lymph_nodes=False,)
data;

############Set parameters for plotting/saving the data#################
data = remove_units(data)
#bounds = data[0.0]['tumor_environment']['dimensions']['bounds']


data_export = open(experiment_dir+'/data_export.pkl', 'wb')
pickle.dump(data, data_export)
data_export.close()

individual_analysis(analysis_dir = all_dir, experiment_id=experiment_id,
                    bounds=bounds, tcells=True, lymph_nodes=False)

In [ ]:
#using experimental CODEX data
data = tumor_tcell_abm(tumors=tumors,
                       tcells=tcells,
                       total_time=6000,
                       bounds=[b*units.um for b in bounds],
                       halt_threshold=6000,
                       emit_step=600,
                       field_molecules=['IFNg'],
                       emitter='timeseries',
                       lymph_nodes=False,)
data;

############Set parameters for plotting/saving the data#################
data = remove_units(data)
#bounds = data[0.0]['tumor_environment']['dimensions']['bounds']


data_export = open(experiment_dir+'/data_export.pkl', 'wb')
pickle.dump(data, data_export)
data_export.close()

individual_analysis(analysis_dir = all_dir, experiment_id=experiment_id,
                    bounds=bounds, tcells=True, lymph_nodes=False)

In [ ]:
df_2HC_exp1['AbsoluteX'] = df_2HC_exp1['AbsoluteX'] + 600
df_2HC_exp1['AbsoluteY'] = df_2HC_exp1['AbsoluteY'] + 600

tumors = {}
IFNg = {}
tcells = {}

for i in range(len(df_2HC_exp1)):
  if df_2HC_exp1.iloc[i, Celltype] == 'PDL1p' or df_2HC_exp1.iloc[i, Celltype] == 'PDL1n':
    tumors.update({
        'tumor_'+ str(df_2HC_exp1.index[i]) : {
            'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
            'type': 'tumor',
            'cell_state': df_2HC_exp1.iloc[i, Celltype],
            'PDL1': df_2HC_exp1.iloc[i, PDL1],
            'MHCI': df_2HC_exp1.iloc[i, H2Kb],
            'diameter': tumor_diameter*units.um,
        }
    })

    IFNg.update({
        df_2HC_exp1.index[i] : {
            'amount': df_2HC_exp1.iloc[i, IFNg_con],
            'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
        }
    })

  else:
    tcells.update({
        'tcell_'+ str(df_2HC_exp1.index[i]) : {
            'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
            'type': 'tcell',
            'cell_state': df_2HC_exp1.iloc[i, Celltype],
            'PD1': df_2HC_exp1.iloc[i, PD1],
            'diameter': tcell_diameter*units.um,
        }
    })
    IFNg.update({
        df_2HC_exp1.index[i]: {
        'amount': df_2HC_exp1.iloc[i, IFNg_con],
        'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
        }
    })

# print the first tcell
tcell_ids = list(tcells.keys())
print(tcells[tcell_ids[0]])

#using experimental CODEX data
data = tumor_tcell_abm(tumors=tumors,
                       tcells=tcells,
                       total_time=10000,
                       bounds=[b*units.um for b in bounds],
                       halt_threshold=6000,
                       emit_step=600,
                       field_molecules=['IFNg'],
                       emitter='timeseries',
                       lymph_nodes=False,)
data;

############Set parameters for plotting/saving the data#################
data = remove_units(data)
#bounds = data[0.0]['tumor_environment']['dimensions']['bounds']


data_export = open(experiment_dir+'/data_export.pkl', 'wb')
pickle.dump(data, data_export)
data_export.close()

individual_analysis(analysis_dir = all_dir, experiment_id=experiment_id,
                    bounds=bounds, tcells=True, lymph_nodes=False)

### Problem #2 (30 points)
What might be some scientific questions you could use this model to answer? What parameters would you set to answer at least one of these questions? (*note- you do not have to actually run these simulations because the parameters you suggest here would probably take too much compute to actually run).

The most interesting questions that can be answered using this ABM and CODEX data are how the tumor environment evolves over time without a cell count halt threshold (though cell count increases very slowly at first but faster with more time). The parameters with which we can tune is quite limited if only give one dataset as a lot of the parameters

### Problem #3 (20 points)
How might starting imported data at (0,0) influence the simulation vs. centering the data? In what case would you want to start at 0,0?

I plotted the data at 0,0 without reading the question thinking it would be a condition that I tune. It seems like the border acts like a wall and that affected the diffusion of IFNg. Both the boundaryh external and internal IFNg graphs looks different with the centered simulation having more internalized IFNg and less boundary IFNg. It is also visually reflected by the simulation animations.

But given a very short time course, neither the # of tumors nor T cells varied between the two. When I increased the simulation time to 6000s, I was able to see variations in the  cytotoxic packet transfer dynamics and individual cells being exposed low and high IFNg concentrations. There wasn't PD1p T cell growth and tumor growth increased.

In the case where there's an cultured tumor and it is pressed right against the wall of its culture container or we are interested in observing contact inhibition behaviours (since cells in the bottom left corner are very restricted), it'd make sense to start the simulation at (0,0). 